# Classification Problem
Next I will develop a Logistic Regression model to predict different classes. More specifically Logistic Regression is used to estimate the probability that an instance, element or observation belongs to a certain class. The use of one of the most popular collections of information for the purpose of classification is the Titanic dataset and the model I have developed will be submitted to Kaggle's 'Titanic' competition.

The purpose of this model is to identify if these passengers 'Survived' or 'Not' which will involve creating a target output column populated with simple binary results of '1' or '0'.

## Import the Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve
%matplotlib inline

## Import the Data

In [ ]:
titanic = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/titanic_data.csv')
titanic.head()

In [ ]:
# check the column names and data types
titanic.info()

So I can determine there are a total of 183 entries in this dataset. Initial thoughts are that it might be worth using a more comprehensive dataset, one which might contain the full list of passengers (1309) rather than just a subset (183). This is the most comprehensive list available for the purpose of this exercise that I can find, although estimates for the total number of passengers and crew members are thought to be in the region of 2220. The most comprehensive datasets might be Encyclopedia Titanica and Wikipedia, both of which can be found online.

In [ ]:
# importing once again
titanic = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/titanic.csv',
                     header=0,
                     names = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin',
                              'Embarked','WikiId','Name_wiki','Age_wiki','Hometown','Boarded','Destination','Lifeboat','Body',
                              'Class'])
titanic.head()

In [ ]:
# check the column names and data types
titanic.info()

In [ ]:
titanic.shape

## Clean the Data
Removing unwanted columns and rows and feature engineering is the next important step. Straight away I can see the second dataset I have imported from Kaggle which I have named 'titanic.csv' has a more comprehensive number of entries but also contains 21 columns as opposed to just 12 in the first set. Time to establish which of these columns will be kept or removed using some dimensionality reduction and combination, before establishing what is to be included in a Pandas DataFrame table and target Series.

I can remove 'PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Embarked', 'WikiId', 'Name_wiki', 'Hometown', 'Destination', 'Lifeboat', 'Body' and 'Class' which will significantly reduce clutter in my table as these features provide no causal relationship with passenger Survival, some of which also represent duplicated information such as passenger class 'Pclass' and 'Class'. This initial step of reductionality helps provide a much more useful dataset overall.

Next, let's determine the index and column values.

In [ ]:
titanic.index

So the index starts at 0 and ends at 1309, a total of 1310 passengers (not including crew members).

In [ ]:
titanic.columns

Creating a variable to store the dropped columns:

In [ ]:
drop_cols = ['PassengerId','Name','Age','Ticket','Fare','Embarked','WikiId','Name_wiki','Hometown','Destination','Lifeboat','Body',
             'Class']

I am using 'Age_wiki' which appears to be a much more comprehensive set of ages from the Wikipedia web site.

In [ ]:
# removing those columns (axis=1) from the titanic dataset inplace (without copying df)
titanic.drop(drop_cols, axis=1, inplace=True)

Which columns or features are left now?

In [ ]:
titanic.columns

## Missing Values
Next it's really important to remove or impute any Null or missing values. This depends on any row values which are missing and also on the data type for each column.

In [ ]:
# create a dict
titanic = pd.DataFrame({
    'Survived': pd.Series(titanic['Survived']),
    'Pclass': pd.Series(titanic['Pclass']),
    'Sex': pd.Series(titanic['Sex']),
    'SibSp': pd.Series(titanic['SibSp']),
    'Parch': pd.Series(titanic['Parch']),
    'Cabin': pd.Series(titanic['Cabin']),
    'Age_wiki': pd.Series(titanic['Age_wiki']),
    'Boarded': pd.Series(titanic['Boarded'])
})

Calculating the total number of missing or Null values across the 'titanic' dataframe gives:

In [ ]:
titanic_missing = pd.isnull(titanic).sum()
print(titanic_missing)

Assessing this output I can determine that the null values in 'Cabin' simply represent tboarding location recorded.hose who did not have a cabin for sleeping quarters. These passengers would have traveled in other areas of the ship so it's important not to drop these values as they represent important data.

There are five null values for the 'Boarded' column so for whatever reason these passengers did not have their boarding locations recorded. It's impossible to really know where these individuals boarded the Titanic so I can either leave the values as Null, or remove each of these five entries.

Finding the complete set of Null values for the entire titanic dataset:

In [ ]:
titanic_null = titanic[titanic.isnull().any(axis=1)]
print(titanic_null)

Taking a look at the total number of Null or missing values for the 'Age_wiki' column only:

In [ ]:
num_age_null = titanic['Age_wiki'].isnull().sum()
print(num_age_null)

And identifying each row in the dataframe which contains a null value for 'Age_wiki' can be achieved as follows:

In [ ]:
titanic[titanic['Age_wiki'].isnull()]

Storing these null values in a variable called age_null as I may need to use these later.

In [ ]:
age_null = titanic[titanic['Age_wiki'].isnull()]
print(age_null)

I can make a decision whether to include these 7 passengers and merely impute some average age for their respective 'Sex', impute an average based on the overall 'Age_wiki', or remove them completely. Seeing as the majority of information for each of these passengers (roughly 4/7ths to 5/7ths) is present I would prefer to keep these entries, so imputing mean values for age based on the individuals sex may be a reasonably accurate average.

To find the overall average age only for those null 'Age_wiki' values:

In [ ]:
titanic.groupby(titanic['Age_wiki'].isnull()).mean()

This overall mean or average may not be as accurate as calculating the average age for both male and female passengers and imputing them into the 7 missing values.

Checking the Null values for the 'Boarded' column:

In [ ]:
titanic[titanic['Boarded'].isnull()]

In [ ]:
titanic.groupby(titanic['Boarded'].isnull()).mean()

### Calculate Average Age
Calculating the average age for male and female passengers in the table can be done by summing each individual age (by sex) and dividing by the total number of male or female passengers respectively.

In [ ]:
# find the number or count of each unique value in the Sex column
titanic['Sex'].value_counts().unique()

Having looked at the source data file it hasn't been split into training or test data yet. The csv file contains labeled data entries up to passenger number 891 but no further. Predicted outcomes need to be applied to the unlabeled passengers from 892 up to 1309 inclusive, so the total above is incorrect. 

This will need to be fixed later on once the data is split into training and test sets.

In [ ]:
import seaborn as sns

titanic.Survived[titanic.Sex == 'male'].value_counts().plot(kind='bar', alpha=0.5)
plt.title("Male Survival")
# create style
sns.set_style("ticks")

In [ ]:
titanic.Survived[titanic.Sex == 'female'].value_counts().plot(kind='bar', alpha=0.5)
plt.title("Female Survival")
# create style
sns.set_style("ticks")

So having determined the unique classes within the 'Sex' column I can further identify the number of Males and Females who survived or not. By taking the 'Survived' column and sub-dividing it according to gender it displays how women were far more likely to have survived the Titanic disaster based on the predictor variables included with this dataset.

Next I want to group each category of male and female and store them in a variable called 'gender'.

In [ ]:
gender = titanic.groupby(titanic['Sex'])

Checking the first few entries for both sexes:

In [ ]:
# Import Library
import seaborn as sns
import matplotlib.pyplot as plt

# Countplot
sns.catplot(x ="Sex", hue ="Survived",
kind ="count", data = titanic)

Now there are two variables, one with all the male and one with all the female passengers in the Titanic dataset. There are a total of 843 male and 466 female passengers.

The next step is to add these totals together.

In [ ]:
male_total = 843
female_total = 466
gender_total = male_total + female_total
gender_total

Summing the total of all ages for all the passengers:

In [ ]:
age = titanic['Age_wiki'].sum()
print(age)

And dividing by the total number of passengers:

In [ ]:
age_ave = age / gender_total
age_ave

So the overall average age for all passengers calculates to just over 29 years old. Using the describe method to check this gives:

In [ ]:
titanic.describe()

So the first item to notice is that only numeric data appears to have been captured which will need to be fixed soon, but the answer I was looking for now, the mean age found under the 'Age-wiki' column is 29.415829 which is close to the value just calculated of 29.258525, but not identical.

Next, to see the average ages for both male and female classes:

In [ ]:
titanic.groupby(by='Sex')['Age_wiki'].mean()

So this produces the mean Age by Sex. What if I wanted to find an average age just for the missing values? I could create a sub-group of null values in the age column and apply the mean method.

In [ ]:
titanic.groupby(titanic['Age_wiki'].isnull()).mean()

In [ ]:
male = gender.get_group('male')
male.head(5)

In [ ]:
female = gender.get_group('female')
female.head(5)

I can see that all the variables are numeric except for the 'Boarded' column which is populated with categorical values. I will change these categorical values into numeric values using 'one-hot encoding'.

Viewing the total number of Male passengers who didn't survive (0.0), or did survive (1.0).

In [ ]:
titanic.loc[titanic.Sex == 'male']['Survived'].value_counts().plot(kind='bar', figsize=(8,5))

The total number of Female passengers who didn't survive (0.0), or did survive (1.0).

In [ ]:
titanic.loc[titanic.Sex == 'female']['Survived'].value_counts().plot(kind='bar', figsize=(8,5))

In [ ]:
gender_num = {'male': 0, 'female': 1}

titanic['Sex'] = titanic['Sex'].map(gender_num)
titanic.head()

The next question is 'What do I do with these average ages?' Because there are only 7 values missing for 'Age' I believe imputing the missing values will be more beneficial than simply deleting the entries altogether. 

I will use the fillna() method to replace any Null values with imputed values for the average age for both men and women.

In [ ]:
titanic['Age_wiki'].fillna()

What about the string and categorical variables in the dataset? An important consideration to make when using visualizations would be the data types involved. For example, information can be split into numeric (quantitative) data and categorical (qualitative) data. Categorical data values could be Binomial (such as the target outcome 'Survived', or 'Sex'), Nominal (such as 'Cabin', or 'Boarded'), perhaps even Ordinal (such as 'Pclass'). 'Age_wiki' contains continuous values and the rest such as 'SibSp' (number of Siblings or Spouse) and 'Parch' (number of Children accompanied by Parents) would be discrete values.

When it comes to visualizing these different types of data it is generally better to use scatter and line plots for numeric data, but for categorical data, frequency distributions, bar charts and histograms may be a better approach for viewing different classes or sub-sets of values.

## Grouping Data Together
Taking a look at the average values for each feature based on their survival.

In [ ]:
titanic.groupby('Survived').mean()

In [ ]:
men = titanic.loc[titanic.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
women = titanic.loc[titanic.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

## Predictor and Target Variables
Now I've established which features are to be included in the whole dataset, it's important to conduct a separation of the predictor variables contained in a dataframe and the target series. This will also lay a foundation for further splitting the labeled data into training and test sets later on.

In [ ]:
# removing the 'Survived' column from the predictors DataFrame variable X
X = titanic.drop(pd.Series(titanic['Survived'], axis=1, inplace=True)
# assigning this dropped column to the target Series variable y
y = pd.Series(titanic['Survived'])

In [ ]:
print(X.head)

In [ ]:
print(y.head)

## Nature of the Data
Of the remaining data 'Survived' is a float which needs to be changed to int. 

In [ ]:
titanic['Survived'] = titanic['Survived'].astype('int8', copy=True)

In [ ]:
titanic.describe()

## Visualizations

In [ ]:
fig = plt.figure(figsize=(18,6))
titanic.Survived.value_counts(normalize=True).plot(kind="bar", alpha=0.5)
plt.show()

Looking into relationships between the different columns can provide more insight, for example between 'Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Boarded' and their 'Survived' status.

In [ ]:
titanic.hist(bins=50, figsize(20,15))
plt.show()

The question is "How do I prepare the dataset with the correct number of total labeled entries?". I can either change the data at source and slice it using Excel, or alternatively slice the data in Python to only include the first 891 passengers. The reason this needs to be done is because of the risk of feeding inaccurate and unlabeled data back into the model. I believe using test data will introduce bias into the classification results. 

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression().fit(iris['data'], iris['target'])
model.predict(iris['data'])

In [ ]:
# Drop all categorical features
cat_feat = ['PassengerId', 'Name', 'Ticket', 'Sex', 'Cabin', 'Embarked']
titanic.drop(cat_feat, axis=1, inplace=True)
titanic.head()

In [ ]:
# Drop all continuous features
cont_feat = ['PassengerId', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Fare']
titanic.drop(cont_feat, axis=1, inplace=True)
titanic.head()

In [ ]:
titanic.groupby('Survived').mean()
titanic.groupby(titanic['Age'].isnull()).mean()

## Plot Continuous Features

In [ ]:
for i in ['Age', 'Fare']:
    died = list(titanic[titanic['Survived'] == 0][i].dropna())
    survived = list(titanic[titanic['Survived'] == 1][i].dropna())
    xmin = min(min(died), min(survived))
    xmax = max(max(died), max(survived))
    width = (xmax - xmin) / 40
    sns.distplot(died, color='r', kde=False, bins=np.arange(xmin, xmax, width))
    sns.distplot(survived, color='g', kde=False, bins=np.arange(xmin, xmax, width))
    plt.legend(['Did not survive', 'Survived'])
    plt.title('Overlaid histogram for {}'.format(i))
    plt.show()

In [ ]:
for i, col in enumerate(['Pclass', 'SibSp', 'Parch']):
    plt.figure(i)
    sns.catplot(x=col, y='Survived', data=titanic, kind='point', aspect=2,)

## Combine SibSp and Parch

In [ ]:
for i, col in enumerate(['SibSp', 'Parch']):
    plt.figure(i)
    sns.catplot(x=col, y='Survived', data=titanic, kind='point', aspect=2, )

In [ ]:
titanic['Family_cnt'] = titanic['SibSp'] + titanic['Parch']

In [ ]:
titanic['family_cnt'] = titanic['SibSp'] + titanic['Parch']
sns.catplot(x='family_cnt', y='Survived', data=titanic, kind='point', aspect=2,)

In [ ]:
titanic['Family_cnt'] = titanic['SibSp'] + titanic['Parch']
titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)
titanic.head()

## Plot Categorical Features

In [ ]:
for i, col in enumerate(['Cabin_ind', 'Sex', 'Embarked']):
    plt.figure(i)
    sns.catplot(x=col, y='Survived', data=titanic, kind='point', aspect=2, )

In [ ]:
titanic.groupby(titanic['Cabin'].isnull()).mean()

In [ ]:
titanic['Cabin_ind'] = np.where(titanic['Cabin'].isnull(), 0, 1)
titanic.head(10)

In [ ]:
titanic.pivot_table('Survived', index='Sex', columns='Embarked', aggfunc='count')

In [ ]:
titanic.pivot_table('Survived', index='Cabin_ind', columns='Embarked', aggfunc='count')

## Create indicator for Cabin

In [ ]:
titanic['Cabin_ind'] = np.where(titanic['Cabin'].isnull(), 0, 1)
titanic.head()

In [ ]:
# Convert Sex to numeric values
gender_num = {'male': 0, 'female': 1}

titanic['Sex'] = titanic['Sex'].map(gender_num)
titanic.head()

In [ ]:
# Drop Cabin and Boarded
titanic.drop(['Cabin', 'Boarded'], axis=1, inplace=True)
titanic.head()

In [ ]:
features = titanic.drop('Survived', axis=1)
labels = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
features = titanic.drop('Survived', axis=1)
labels = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset) / len(labels), 2))

In [ ]:
X_train.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_features.csv', index=False)
X_val.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_features.csv', index=False)
X_test.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_features.csv', index=False)

y_train.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_labels.csv', index=False)
y_val.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_labels.csv', index=False)
y_test.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_labels.csv', index=False)

In [ ]:
print(len(labels), len(y_train), len(y_val), len(y_test))

In [ ]:
# Write out cleaned data
titanic.to_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/titanic_cleaned.csv', index=False)

## Fit and evaluate a basic model using 5-fold Cross-Validation

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

tr_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_features.csv')
tr_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_labels.csv', header=None)

In [ ]:
rf = RandomForestClassifier()

scores = cross_val_score(rf, tr_features, tr_labels.values.ravel(), cv=5)
scores

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 100],
    'max_depth': [2, 10, 20, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

## Pipeline: Evaluate results on validation set
Using the Titanic dataset from this Kaggle competition.

In this section, we will use what we learned in last section to fit the best few models on the full training set and then evaluate the model on the validation set.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

tr_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_features.csv')
tr_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_labels.csv', header=None)

val_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_features.csv')
val_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_labels.csv', header=None)

te_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_features.csv')
te_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_labels.csv', header=None)

In [ ]:
rf1 = RandomForestClassifier(n_estimators=5, max_depth=10)
rf1.fit(tr_features, tr_labels.values.ravel())

rf2 = RandomForestClassifier(n_estimators=100, max_depth=10)
rf2.fit(tr_features, tr_labels.values.ravel())

rf3 = RandomForestClassifier(n_estimators=100, max_depth=None)
rf3.fit(tr_features, tr_labels.values.ravel())

In [ ]:
for mdl in [rf1, rf2, rf3]:
    y_pred = mdl.predict(val_features)
    accuracy = round(accuracy_score(val_labels, y_pred), 3)
    precision = round(precision_score(val_labels, y_pred), 3)
    recall = round(recall_score(val_labels, y_pred), 3)
    print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(mdl.max_depth,
                                                                         mdl.n_estimators,
                                                                         accuracy,
                                                                         precision,
                                                                         recall))

1. Explore and clean the data
2. Split data into train / validation / test
3. Fit an initial model and evaluate
4. Tune hyper parameters
5. Evaluate on validation set
6. Final model selection and evaluation on test set

In [ ]:
y_pred = rf2.predict(te_features)
accuracy = round(accuracy_score(te_labels, y_pred), 3)
precision = round(precision_score(te_labels, y_pred), 3)
recall = round(recall_score(te_labels, y_pred), 3)
print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(rf2.max_depth,
                                                                     rf2.n_estimators,
                                                                     accuracy,
                                                                     precision,
                                                                     recall))